In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.typing import NDArray
from PIL import Image
from transformers import pipeline
import trimesh

In [ ]:
depth_estimator = pipeline(
    "depth-estimation", model="depth-anything/Depth-Anything-V2-Small-hf"
)

In [ ]:
image_dim = 512

In [ ]:
image_path = "example_img.png"
original_image = Image.open(image_path).convert("RGB")
original_image = original_image.resize((image_dim, image_dim))

display(original_image)

In [ ]:
# Estimate depth
depth_result = depth_estimator(original_image)
depth_array = np.array(depth_result["depth"])

# Explicit normalization (model may produce unnormalized output)
depth_norm = (depth_array - depth_array.min()) / (depth_array.max() - depth_array.min())

# Visualize normalized depth with cutoff applied
plt.imshow(depth_norm, cmap="gray")
plt.title("Normalized Depth")
plt.axis("off")
plt.show()

In [ ]:
# Configurable cutoff threshold
depth_threshold = 0.5  # between 0 and 1
depth_cutoff = np.clip(depth_norm, depth_threshold, 1.0)
depth_cutoff -= 0.5
depth_cutoff /= 0.5

# Visualize normalized depth with cutoff applied
plt.imshow(depth_cutoff, cmap="gray")
plt.title("Normalized Depth with Cutoff")
plt.axis("off")
plt.show()

In [ ]:
# Optionally save depth image to greyscale png for hand-editing
# array_16_bit = (depth_cutoff * 65535).astype(np.uint16)
# image = Image.fromarray(array_16_bit, mode='I;16')
# image.save("downsampled_heightmap.png")

In [ ]:
# Optionally reload greyscale depth image post-editing
# edited_image = Image.open("downsampled_heightmap.png").convert('I;16')
# loaded_array_16_bit = np.array(edited_image, dtype=np.float32)
# depth_cutoff = loaded_array_16_bit / 65535.0

In [ ]:
# Find coordinates of non-black (valid) pixels
nonzero_coords = np.argwhere(depth_cutoff > 0.0)

# Compute tight bounding rectangle around non-black pixels
y_min, x_min = nonzero_coords.min(axis=0)
y_max, x_max = nonzero_coords.max(axis=0)

# Width and height of bounding rectangle with buffer
width = x_max - x_min
height = y_max - y_min

# Determine side length of square (largest dimension)
side_length = max(width, height)

# Center of buffered bounding box
center_x = (x_min + x_max) // 2
center_y = (y_min + y_max) // 2

# Compute square coordinates, ensuring bounds stay within image dimensions
half_side = side_length // 2
new_x_min = max(center_x - half_side, 0)
new_x_max = min(center_x + half_side, depth_cutoff.shape[1])
new_y_min = max(center_y - half_side, 0)
new_y_max = min(center_y + half_side, depth_cutoff.shape[0])

# Perform cropping
cropped_depth = depth_cutoff[new_y_min:new_y_max, new_x_min:new_x_max]

# Display cropped depth map with buffer
plt.imshow(cropped_depth, cmap="gray")
plt.title("Cropped Depth Map")
plt.axis("off")
plt.show()

In [ ]:
# Downsample to desired resolution
resolution = 128  # Configurable
downsampled_depth = np.array(
    Image.fromarray(cropped_depth).resize(
        (resolution, resolution), resample=Image.BILINEAR
    )
)

# Visualize the downsampled depth map
plt.imshow(downsampled_depth, cmap="gray")
plt.title("Downsampled Depth Map")
plt.axis("off")
plt.show()

In [ ]:
def generate_enclosed_heightfield(
    height_map: NDArray, h_spacing: float = 1.0, height_scale: float = 1.0
) -> trimesh.Trimesh:
    """Create a fully enclosed height map mesh from a normalized value height nap

    Args:
        height_map: 2D height map array
        h_spacing: Horizontal spacing between adjacent vertices
        height_scale: Scaling to apply to the height values

    Returns:
        Fully enclosed Trimesh of the height map
    """
    height_map = np.flipud(height_map)
    input_rows, input_cols = height_map.shape

    # Add outer ring
    grid_rows, grid_cols = input_rows + 2, input_cols + 2

    # Generate grid vertices with outer ring
    x = np.linspace(0, (grid_cols - 1) * h_spacing, grid_cols)
    y = np.linspace(0, (grid_rows - 1) * h_spacing, grid_rows)
    xv, yv = np.meshgrid(x, y)

    # Create expanded height map with outer vertices at 0 height
    height = np.zeros((grid_rows, grid_cols))
    height[1:-1, 1:-1] = height_map * height_scale

    vertices = np.column_stack([xv.ravel(), yv.ravel(), height.ravel()])

    # Calculate vertex indices for each quad
    row_indices, col_indices = np.indices((grid_rows - 1, grid_cols - 1))
    base_indices = row_indices * grid_cols + col_indices

    # Correct winding order for top surface faces
    faces = np.column_stack(
        [
            base_indices.ravel(),
            (base_indices + 1).ravel(),
            (base_indices + grid_cols).ravel(),
            (base_indices + 1).ravel(),
            (base_indices + grid_cols + 1).ravel(),
            (base_indices + grid_cols).ravel(),
        ]
    ).reshape(-1, 3)

    # Add a central vertex at the base center
    center_base_vertex = np.array([[np.mean(x), np.mean(y), 0]])
    vertices = np.vstack([vertices, center_base_vertex])
    center_vertex_index = vertices.shape[0] - 1

    # Identify boundary vertices (outer ring)
    top_row = np.arange(grid_cols)
    bottom_row = np.arange(grid_cols * (grid_rows - 1), grid_cols * grid_rows)
    left_column = np.arange(grid_cols, grid_cols * (grid_rows - 1), grid_cols)
    right_column = np.arange(2 * grid_cols - 1, grid_cols * (grid_rows - 1), grid_cols)

    boundary_indices = np.concatenate(
        [top_row, right_column, bottom_row[::-1], left_column[::-1]]
    )

    # Create bottom faces efficiently (inverted winding)
    bottom_faces = np.column_stack(
        [
            np.roll(boundary_indices, -1),
            boundary_indices,
            np.full(len(boundary_indices), center_vertex_index),
        ]
    )

    # Combine top and bottom faces
    faces = np.vstack([faces, bottom_faces])

    # Create and return the watertight mesh
    return trimesh.Trimesh(vertices=vertices, faces=faces)

In [ ]:
mesh = generate_enclosed_heightfield(
    height_map=downsampled_depth, h_spacing=0.16, height_scale=14.0
)

In [ ]:
mesh.show()

In [ ]:
# Optional - subtract a thin box to remove any flat parts of the height field
box_size = [
    mesh.bounds[1][0] * 1.05,
    mesh.bounds[1][0] * 1.05,
    0.05,
]  # x, y, z dimensions

# Create the box mesh
box = trimesh.creation.box(extents=box_size)
mesh_center_x = (mesh.bounds[0, 0] + mesh.bounds[1, 0]) / 2
mesh_center_y = (mesh.bounds[0, 1] + mesh.bounds[1, 1]) / 2
box.apply_translation((mesh_center_x, mesh_center_y, 0.0))
final_mesh = trimesh.boolean.difference([mesh, box])
final_mesh.show()

In [ ]:
final_mesh.export("no_background.stl")
pass

In [ ]:
# Optional - attach a cylinder background
# Calculate cylinder radius as half-diagonal to fully enclose square face mesh
mesh_h_size = mesh.bounds[1, 1]
cylinder_radius = np.sqrt(2 * ((mesh_h_size / 2) ** 2)) * 1.05  # +5% margin
cylinder_height = 1.0  # configurable thickness in mm
cylinder = trimesh.creation.cylinder(
    radius=cylinder_radius, height=cylinder_height, sections=64
)

# Position cylinder behind mesh clearly below the lowest Z-value of mesh
mesh_center_x = (mesh.bounds[0, 0] + mesh.bounds[1, 0]) / 2
mesh_center_y = (mesh.bounds[0, 1] + mesh.bounds[1, 1]) / 2
cylinder_z_offset = mesh.bounds[0, 2] - (cylinder_height / 2)

# Translate cylinder to correct position
cylinder.apply_translation((mesh_center_x, mesh_center_y, cylinder_z_offset))

# Combine face mesh with cylinder backing
# final_mesh = trimesh.util.concatenate([tri_mesh, cylinder])
final_mesh = trimesh.boolean.union([mesh, cylinder])
final_mesh.show()

In [ ]:
final_mesh.export("cylinder_background.stl")
pass